Sometimes your input data can be nested with more difficult structure than a simple table or a matrix.

In such cases it is sometime useful to shift mental orientation to analyze and extract information froms rows rather then non-defined columns.

In [1]:
from utils import css_from_file
css_from_file('style/style.css');

In [2]:
#!pip install nltk

In [3]:
import json
import numpy as np
from pprint import pprint
from nltk import download, word_tokenize

download('punkt')

[nltk_data] Downloading package punkt to /home/batman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
with open("data/companies/companies.json") as dataf:
    data = [json.loads(line) for line in dataf]

An example of deeply nested data with various data types:

Exercise:

1. Name variable types
2. What do you do with lists, geo location?
3. What do you do with counts?

In [5]:
this_json_row = data[7]
pprint(this_json_row)

{'address': {'city': 'Seattle',
             'country': 'United States',
             'postalCode': '98134',
             'raw': '624 South Lander St\n'
                    'Suite 28\n'
                    'Seattle,\n'
                    'WA\n'
                    '98134\n'
                    'United States',
             'region': 'WA',
             'street': '624 South Lander St'},
 'description': 'At 36th avenue design|build we are committed to total client '
                'satisfaction. We believe that strong and lasting '
                'relationships built on integrity and trust, earned through '
                'the remodel, is as important as the renovation of your home. '
                'We consider every project an opportunity to participate with '
                'our clients in a unique and artful, design and construction '
                'process. To each of our clients, our commitment remains '
                'consistent: concise communication, integrity, and prid

In [6]:
#Does this work???
from pandas.io.json import json_normalize
new_json = json_normalize(this_json_row).to_json()
#pprint(new_json)

In [7]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
import logging.config
print(ABSOLUTE_LOGGING_PATH)
logging.config.fileConfig(ABSOLUTE_LOGGING_PATH)
myLogger = logging.getLogger()
myLogger.setLevel("DEBUG")


In [68]:
class TechnologyFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        # SEND IN PARAMETERS FOR TRANSFOMER
        # DO NOT SEND IN DATA HERE
        #self.data = data
        #print(data)
        print(self.__class__.__name__,"Init ")
        
    def fit(self, x, y = None): 
        return self
    
    def transform(self,data):
        print("Transforming",len(data),"rows")
        out = list()
        for row in data:
            if 'technologies' in row.keys():
                these_techs=[name for name in row['technologies']]
                these_tech_dicts = dict((tech, 1) for tech in these_techs)
                if 0:
                    print("Found",len(these_tech_dicts), "techs")
            else:
                these_tech_dicts = dict()
            out.append(these_tech_dicts)
        return out

tech_trf = TechnologyFeatures()

tech_trf.transform(data[0:9]);

TechnologyFeatures Init 
Transforming 9 rows


In [79]:
class AddressFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        print(self.__class__.__name__,"Init")
        
    def fit(self, x, y = None): 
        return self
    
    def transform(self,this_data):
        for i,row in enumerate(this_data):
            res = row.get('address',{}).get('city',None)
            if not res:
                res = row.get('extension',{}).get('address',{})
                #pprint(res)
                if not res:
                    #pprint(row)
                    #print('NOPE')
                    #raise
                    res = None
            
            print(i,res)

        return self

address_trf = AddressFeatures()
address_trf.transform(data[0:100]);


AddressFeatures Init
0 Auckland, New Zealand
1 Cyprus
2 Huntsville, Alabama Area
3 Düsseldorf
4 Nijmegen Area, Netherlands
5 Ukraine
6 Sydney Area, Australia
7 Seattle
8 Kolding
9 Greater Minneapolis-St. Paul Area
10 Philippines
11 Barcelona
12 United States
13 Saint Cloud, Florida
14 Colorado Springs, Colorado Area
15 Malta
16 Liverpool
17 None
18 None
19 Austin, Texas
20 None
21 None
22 Johannesburg Area, South Africa
23 Santa Barbara, California
24 Chester, United Kingdom
25 Saint-Jerome, Quebec, Canada
26 Aberdeen, United Kingdom
27 Hull, United Kingdom
28 Greater Boston Area
29 Dallas/Fort Worth Area
30 Greater Los Angeles Area
31 Greater Atlanta Area
32 Goiânia, Goiás, Brazil
33 None
34 Calgary, Alberta, Canada
35 Reading, United Kingdom
36 London, United Kingdom
37 Canoga Park
38 Washington D.C. Metro Area
39 San Antonio, Texas Area
40 Palm Springs, California
41 Brno
42 Cotia, São Paulo, Brazil
43 None
44 United States
45 Mumbai Area, India
46 Jakarta
47 İstanbul
48 None
49 Gen

In [30]:
class ExtractDescription(BaseEstimator, TransformerMixin):
    def __init__(self,data):
        self.data = data
        print(self.__class__.__name__,"Init with size",len(data))
        
    def fit(self, x, y = None): 
        return self
    
    def transform(self):

        return self
description_trf = ExtractDescription(data[7:9])
description_trf.transform();

ExtractDescription Init with size 2


In [57]:
#%%timeit
#this_sample = data[0:10]

In [49]:
# TEST ADDRESS
feat_trf = AddressFeatures(this_sample[7])
#pprint(feat_trf)
feat_trf.transform()

Init with size 16
{'city': 'Seattle',
 'country': 'United States',
 'postalCode': '98134',
 'raw': '624 South Lander St\nSuite 28\nSeattle,\nWA\n98134\nUnited States',
 'region': 'WA',
 'street': '624 South Lander St'}


AddressFeatures(data={'domain': '36designbuild.com', 'uuid': '377494fe2c345c77806c89a9a9c5653f', 'public_url': 'http://www.linkedin.com/company/6629242', 'name': '36th Avenue Design Build', 'website': 'http://www.36designbuild.com', 'industries': ['Construction'], 'description': 'At 36th avenue design|build we are ...tion_triple': {'city': 'Seattle', 'administrative_area': 'Washington', 'country': 'United States'}}})

Exercise
--------------

Write a pipeline to transform company records.
1. Select 3 types of features you want to transform (like descrpition, list of skills, technologies, address etc)
2. Create a pipeline in this format:
```python
make_union(
    make_pipeline(TechnologyFeatures(), DictVectorizer()),
    make_pipeline(AddressFeatures(), DictVectorizer()),
    make_pipeline(ExtractDescription(), CountVectorizer())
)
```
3. Classify industry (like in the previous exercise)

Exercise
===============

1. Write a transformation class called SparsityFilter that accepts a minimum frequency. Watch out for fit function - this class has some state that you must save

```
class SparsityFilter(BaseEstimator, TransformerMixin):
    def __init__(self, min_nnz=None):
        self.min_nnz = min_nnz

    def fit(self, X, y=None):
        ???
        return self

    def transform(self, X):
        return ???
```

In [61]:
# write sparsity class here

Double click to see the solution 

<div class="spoiler">

class SparsityFilter(BaseEstimator, TransformerMixin):
    def __init__(self, min_nnz=None):
        self.min_nnz = min_nnz

    def fit(self, X, y=None):
        self.sparsity = X.getnnz(0)
        return self

    def transform(self, X):
        return X[:, self.sparsity >= self.min_nnz]
</div>